In [ ]:
library(spatstat)
library(dplyr)
#library(ggplot2)
library(imager)
library(stringr)
#ulimit::memory_limit(6000)

In [ ]:
mydata <- read.csv("./data/20220420_JP-TMAs_IMC-TMAs_MIBI_CombinedCelltypes_all.csv")
mydata <- mydata[grep("JP-TMA", mydata$X), ]
mydata[,"DAPI_X"] = mydata[,"DAPI_X"]/0.325
mydata[,"DAPI_Y"] = mydata[,"DAPI_Y"]/0.325
tail(mydata)

In [ ]:
#read in ROIS
myrois <- read.csv("JP-TMAs_ROIs_full.csv")
head(myrois)

In [ ]:
#simplify to tumor
mydata[,'leiden'][mydata[,'leidencelltype5'] == 'epithelial'] <- 'tumor'

In [ ]:
mydata[,'leiden'][mydata[,'leiden'] == 'CD8 T cell'] <- 'CD3 T cell'

In [ ]:
mydata[,'leiden'][mydata[,'leiden'] == 'CD4 T cell'] <- 'CD3 T cell'

In [ ]:
ls_scene = unique(mydata$slide_scene)

In [ ]:
ls_scene

In [ ]:
#### K cross
ls_cells = unique(mydata$leiden) #c('T cell','B cell')#,'tumor','Endothelial','CD68+ immune','fibroblast','CD45 low immune'
ls_from = unique(mydata$leiden) #c('T cell','B cell','tumor')#
#ls_cells = c('tumor','Quies. str.','Vim+ FB','Macrophage','endothelial','CD3 T cell')#,'CD4 T cell','CD8 T cell''FN+ FB',
#ls_from = c('tumor','CD3 T cell') #,'CD4 T cell','CD8 T cell'
combined_df_Kcross <- data.frame(r=double(),
                    theo=double(),
                    iso=double(),
                    slide_scene=character(),
                    cell=character())
combined_df_Kest <- data.frame(r=double(),
                    theo=double(),
                    iso=double(),
                    slide_scene=character(),
                    cell=character())
combined_df_density <- data.frame(d=double(),
                    slide_scene=character(),
                    cell=character())

combined_df_Gcross <- data.frame(d=double(),
                    slide_scene=character(),
                    cell=character())

# loop version 1
datalist = list()
for (s_scene in ls_scene) { #ls_scene[1:18]
  print(s_scene)
  s_scene_scene <- str_replace(s_scene, '_scene', '-Scene-')
  #new - use square ROI
  myroiSubset <- myrois[grep(s_scene_scene, myrois$X),]
  ymin <- myroiSubset %>% pull('ymin')
  ymax <- myroiSubset %>% pull('ymax')
  xmin <- myroiSubset %>% pull('xmin')
  xmax <- myroiSubset %>% pull('xmax')
  #make smaller square
  mydataSubset <- mydata[grep(s_scene, mydata$X), c('X','DAPI_X','DAPI_Y','leiden')]
  mydataSubsetRoi <- filter(mydataSubset, DAPI_X>xmin & DAPI_X<xmax & DAPI_Y>ymin & DAPI_Y<ymax)
  mypattern <- ppp(mydataSubsetRoi[,"DAPI_X"],mydataSubsetRoi[,"DAPI_Y"],c(xmin,xmax),c(ymin,ymax))
  win_p <- mypattern$window
  marks(mypattern) <- factor(mydataSubsetRoi$leiden)
  startTime <- Sys.time()
  for (s_cell in intersect((unique(mydataSubsetRoi$leiden)),ls_cells)) {
      print(s_cell)
      for (s_center in intersect((unique(mydataSubsetRoi$leiden)),ls_from)) {
          print(s_center)
          if (s_cell!=s_center){
              print(s_cell)
              print(s_center)
              # kcross
              Kcross_cell <- Kcross(X=mypattern, from=s_center, to=s_cell,correction='isotropic',rmax=100/.325,nlarge=3000)
              Kcross_cell['slide_scene'] = s_scene
              Kcross_cell['from_cell'] = s_center
              Kcross_cell['to_cell'] = s_cell
              combined_df_Kcross <- rbind(combined_df_Kcross, Kcross_cell)
              #g cross
              Gcross_cell <- Gcross(X=mypattern, i=s_center, j=s_cell, correction=c("rs", "km", "han"))
              Gcross_cell['slide_scene'] = s_scene
              Gcross_cell['from_cell'] = s_center
              Gcross_cell['to_cell'] = s_cell
              combined_df_Gcross <- rbind(combined_df_Gcross, Gcross_cell)
              }
          else {
            mydata_cell = mydataSubsetRoi[grep(s_cell, mydataSubsetRoi$leiden), ]
            mypatternK <- ppp(x=(mydata_cell[,"DAPI_X"]),y=((mydata_cell[,"DAPI_Y"])),window=win_p)
            Kest_cell <- Kest(mypatternK,correction = 'isotropic',rmax=100/.325,nlarge=3000)
            #fit <- ppm(mypatternK, ~ polynom(x,y,2), Poisson())
            #lambda <- predict(fit, locations=mypatternK, type="trend")
            #Kest_cell <- Kinhom(mypatternK,lambda,correction = 'isotropic',rmax=100/.325,nlarge=4000) #inhom
            Kest_cell['slide_scene'] = s_scene
            Kest_cell['cell'] = s_cell
            combined_df_Kest <- rbind(combined_df_Kest, Kest_cell)
            D <- intensity(mypatternK)
            combined_df_density[nrow(combined_df_density) + 1,] <- list(D, s_scene, s_cell)
             }
          }
      
  }
  endTime <- Sys.time()
  print(endTime - startTime)
  #break
    }


In [ ]:
# #save out
# s_num = 'ROI'#'inhom' #first
# write.csv(combined_df_Kcross,sprintf("./data/cycIF_%s_Kcross.csv",s_num), row.names = FALSE)
# write.csv(combined_df_Kest,sprintf("./data/cycIF_%s_Kest.csv",s_num), row.names = FALSE)
# write.csv(combined_df_density,sprintf("./data/cycIF_%s_density.csv",s_num), row.names = FALSE)
# write.csv(combined_df_Gcross,sprintf("./data/cycIF_%s_Gcross.csv",s_num), row.names = FALSE)

In [ ]:
print('done')

## quadrat counts

https://rdrr.io/github/spatstat/spatstat.geom/man/quadratcount.html

In [ ]:
makeQuadrats <- function(p, squareLength) {
  require(spatstat)
  
  dx <- Window(p)$xrange[2] - Window(p)$xrange[1]
  dy <- Window(p)$yrange[2] - Window(p)$yrange[1]
  
  nx <- floor(dx/squareLength)
  ny <- floor(dy/squareLength)
  return(c(nx,ny))
 }

In [ ]:
#### occupancy counts
mydata[,'leiden2'] = sub("$","_",mydata$leiden)
squareLength=60
combined_df_occ <- data.frame(unlist.result_table.=double(),
                    slide_scene=character())
# loop 
pList = list()
for (s_scene in ls_scene) { #ls_scene[1:18]
  print(s_scene)
  s_scene_scene <- str_replace(s_scene, '_scene', '-Scene-')
  #new - use square ROI
  myroiSubset <- myrois[grep(s_scene_scene, myrois$X),]
  ymin <- myroiSubset %>% pull('ymin')
  ymax <- myroiSubset %>% pull('ymax')
  xmin <- myroiSubset %>% pull('xmin')
  xmax <- myroiSubset %>% pull('xmax')
  #make smaller square
  mydataSubset <- mydata[grep(s_scene, mydata$X), c('X','DAPI_X','DAPI_Y','leiden2')]
  mydataSubsetRoi <- filter(mydataSubset, DAPI_X>xmin & DAPI_X<xmax & DAPI_Y>ymin & DAPI_Y<ymax)
  mypattern <- ppp(mydataSubsetRoi[,"DAPI_X"],mydataSubsetRoi[,"DAPI_Y"],c(xmin,xmax),c(ymin,ymax))
  win_p <- mypattern$window
  marks(mypattern) <- factor(mydataSubsetRoi$leiden2)
  nxny <- makeQuadrats(mypattern,50/.325)
  result_table <- quadratcount.splitppp(split(mypattern), nx=nxny[1], ny=nxny[2])
  counts <- data.frame(unlist(result_table))
  if (length(counts) > 0) {
    counts['slide_scene'] = s_scene
    combined_df_occ <- rbind(combined_df_occ, counts)
    }
  
  #break
  }

# #save out
s_num = 'ROI'#'inhom' #first
write.csv(combined_df_occ,sprintf("./data/cycIF_%s_Occ.csv",s_num), row.names = TRUE)
print('done')

## getis ord
https://www.rdocumentation.org/packages/ecespa/versions/1.1-12/topics/getis


Beyond immune density: critical role of spatial heterogeneity in estrogen receptor-negative breast cancer
Sidra Nawaz 1 2 3, Andreas Heindl 1 2 3, Konrad Koelble 4 5, Yinyin Yuan 1 2 3


s=50, 100 or 250 μm.

1.
Cancer hotspot fractional: the fraction of cancer hotspots that are also immune hotspots (fC);

2.
Immune hotspot fractional: the fraction of immune hotspots that are also cancer hotspots (fI); and

3.
Cancer–immune hotspot co-localization: the fractional area of tumor where cancer and immune hotspots were co-localized (fC–I).


second or fourth order

In [ ]:
#shared code
#filter(df, gender == 'M' & id >11)
# Return occupancies and box counts for a list of point patterns
# (spatstat class ppp).
# typesToCount is the phenotype of interest,
# categoriesToCount is the tissue type of interest (e.g., cancer, stroma).
OccsAndCountsPatternList <- function(pList, typesToCount=NULL,
                                     categoriesToCount="All") {
  require(spatstat)
  require(gtools)
  require(pbapply)
  
  OccsCounts <- lapply(pList, function(x){
    getBoxOccsAndCounts(x, typesToCount=typesToCount,
                        categoriesToCount=categoriesToCount)
  })
  
  return(OccsCounts)
}

# Return occupancies and box counts for a single point pattern
# (spatstat class ppp).
# countSizes is the range of box sizes in microns.
# Result is a data frame where each column corresponds to one box size,
# row 1 contains the occupancies (value between 0 and 1),
# and row 2 contains the box counts (integer).
# For fractal dimension at different ranges, call getDimRangeFromSlope
# on the box counts (row 2 of the result).
getBoxOccsAndCounts <- function(p, countSizes=seq(10,600,5),
                                typesToCount=NULL,
                                categoriesToCount="All", thin=NULL) {
  require(pbapply)
  require(spatstat)
  
  print("in gBOAC")
  
  if (!is.null(thin)) {
    p <- randomThinPheno(p, x=thin, typesToCount=typesToCount)
  }
  
  counts <- pblapply(countSizes, function(x){
    res=chopToQuadratsBoth(p, x, typesToCount,
                           categoriesToCount);
    return(res)
  })
  
  counts <- data.frame(matrix(unlist(counts), nrow=2))
  colnames(counts) <- as.character(countSizes)
  
  return(counts)
}


# Performs a quadrat count of the given point pattern with a given
# quadrat size.
chopToQuadratsBoth <- function(p, squareLength=60, typesToCount=NULL,
                               categoriesToCount="All",
                               thinFactor=NULL) {
  require(spatstat)
  require(spatstat.utils)
  require(pbapply)
  
  xmin <- Window(p)$xrange[1]
  xmax <- Window(p)$xrange[2]
  ymin <- Window(p)$yrange[1]
  ymax <- Window(p)$yrange[2]
  dx <- xmax-xmin
  dy <- ymax-ymin
  
  dxTrunc <- 2*squareLength * as.integer(dx / (2*squareLength))
  dyTrunc <- 2*squareLength * as.integer(dy / (2*squareLength))
  
  #print(paste("Total cells: ", p$n))
  
  p <- p[owin(c(xmin, xmin+dxTrunc),c(ymin,ymin+dyTrunc))]
  
  pOrig <- p
  
  if (categoriesToCount=="Tumor" | categoriesToCount=="Cancer" |
      categoriesToCount == "Epithelial")
    p <- subset(p, Tissue.Category %in% list("Tumor", "CancerIsland",
                                             "CancerIslands",
                                             "Cancer Islands",
                                             "epithelial",
                                             "Epithelial", "Epithelium",
                                             "Cancer", "cancer island"))
  else if (categoriesToCount=="Stroma")
    p <- subset(p, Tissue.Category %in% list("Stroma",
                                             "stroma",
                                             "Epithelial associated"))
  
  pSubset <- p
  
  # Keep only the cells of the types we're interested in.
  if (!is.null(typesToCount) & is.null(thinFactor)) {
    pSubset <- subset(p, select=Phenotype)
    pSubset <- subset(pSubset, marks %in% typesToCount)
    #print(paste("After cell filtering: ", pSubset$n))
  }
  if (!is.null(thinFactor) & is.null(typesToCount)) {
    pSubset <- randomThin(pSubset, thinFactor)
    #print(paste("After thinning (no filtering): ", pSubset$n))
  }
  
  # Split the starting pattern into smaller squares.
  # There must be a way to avoid doing this twice.
  # Why would masterCounts have a different length than allCellsCounts here?
  #print("Splitting into squares")
  #masterCounts <- parallelQuad(pSubset, squareLength)
  
  masterCounts <- makeQuadrats(pSubset, squareLength)
  # Split the original pattern (all cell types) into smaller squares.
  #allCellsCounts <- parallelQuad(pOrig, squareLength)
  allCellsCounts <- makeQuadrats(pOrig, squareLength)
  
  #print("Quadrats made.")
  
  # Note which ones are empty.
  toKeep <- sapply(allCellsCounts, function(x){x!=0})
  #print(paste("Omitting", length(toKeep)-sum(toKeep), "of", length(toKeep)))
  #print(paste("Before subsetting:", length(masterCounts), length(allCellsCounts)))
  
  # Get rid of squares corresponding to no tissue.  
  allCellsCounts <- subset(allCellsCounts, toKeep)
  masterCounts <- masterCounts[toKeep]  
  #print(paste("After subsetting:", length(masterCounts), length(allCellsCounts)))
  # If there isn't a valid matrix, put in an empty one.
  
  # Make the counts binary: 1 if there's at least one cell of interest, 0 if not.
  #print("Binarizing grid counts")
  squareCounts <- lapply(masterCounts, function(x){ifelse(x>=1,1,0)})
  squareCounts <- unlist(squareCounts)
  
  return(c(sum(squareCounts)/length(squareCounts), sum(squareCounts)))
  
}

# Create quadrats given a point pattern and a square size.
makeQuadrats <- function(p, squareLength) {
  require(spatstat)
  require(pbapply)
  
  dx <- Window(p)$xrange[2] - Window(p)$xrange[1]
  dy <- Window(p)$yrange[2] - Window(p)$yrange[1]
  
  nx <- floor(dx/squareLength)
  ny <- floor(dy/squareLength)
  
  # Delete a point for potential edge case.
  if (max(p$y) - Window(p)$yrange[2] < 1e-10) {
    #print("edge case detected")
    #print(paste(max(p$y), Window(p)$yrange[2], max(p$y)-Window(p)$yrange[2]))
    p <- subset(p, y<max(p$y))
    #print(max(p$y))
  }
  
  #print(paste(max(p$x), Window(p)$xrange[2], max(p$x)-Window(p)$xrange[2]))
  if (max(p$x) - Window(p)$xrange[2] < 1e-10) {
    #print("edge case detected")
    #print(paste(max(p$x), Window(p)$xrange[2], max(p$x)-Window(p)$xrange[2]))
    p <- subset(p, x<max(p$x))
    #print(max(p$x))
  }
  
  #print(paste(squareLength, ": Splitting", p$n, "points into", nx, "by",
  # ny, "quadrats"))
  
  # If there are too few points, quadratcount.ppp can give an error:
  # Error in table(list(y=yg, x=xg)): all arguments must have the same length
  if (p$n > 0) {
    patternList <-quadratcount.ppp(p, nx, ny)
    #print("Counted")
    #print(length(unlist(patternList, recursive=TRUE)))
    patternList<-unlist(patternList, recursive=FALSE)
  }
  
  else(patternList <- rep(0, nx*ny))
  
  return(patternList)  
}

# Given the box counts and the range of box size, determine the fractal
# dimensions at different length scale ranges.
# boxCounts contains integers corresponding to the number of
# boxes needed to cover the cells of interest at a given box size.
getDimRangeFromSlope <- function(boxCounts, BoxSizes=seq(10,600,5)) {
  # Overall constant to scale the x-axis.
  LongSide <- 7000
  
  if(length(boxCounts) != length(BoxSizes)) {
    print(length(boxCounts))
    print(length(BoxSizes))
    stop("Vector lengths do not match")
  }
  
  if (length(boxCounts[boxCounts<=1])>2 ) {
    res=data.frame(t(rep(0,8)))
    colnames(res) <- c("Small", "Overall", "Large", "Slope_50_100", "Slope_50_200",
                       "Slope_20_80", "Slope_10_80", "Slope_10_40")
    return(res)
  }
  
  x0 <- 1/BoxSizes
  x <- log(LongSide/BoxSizes)
  y <- unlist(log(as.numeric(unlist(boxCounts[1:length(BoxSizes)]))))
  
  #print(rbind(x,y))
  
  if(sum(is.infinite(y)) > 0) {
    warning("Inf replaced with 0")
  }
  
  y[is.infinite(y)] = 0
  
  l <- length(BoxSizes)
  dl <- 80
  
  OverallSlope <- (lsfit(x, y)$coef[2])
  SmallSlope <- lsfit(x[1:2], y[1:2])$coef[2] # 10 to 15
  LargeSlope <- lsfit(x[abs(l-dl):l], y[abs(l-dl):l])$coef[2] # 200 to 600
  
  Slope_50_100 <- lsfit(x[9:19], y[9:19])$coef[2]
  Slope_50_200 <- lsfit(x[9:29], y[9:29])$coef[2]
  Slope_20_80 <- lsfit(x[3:15], y[3:15])$coef[2]
  Slope_10_80 <- lsfit(x[1:15], y[1:15])$coef[2]
  Slope_10_40 <- lsfit(x[1:7], y[1:7])$coef[2]
  
  res <- data.frame(SmallSlope, OverallSlope, LargeSlope, Slope_50_100, Slope_50_200,
                    Slope_20_80, Slope_10_80, Slope_10_40)
  colnames(res) <- c("Small", "Overall", "Large", "Slope_50_100", "Slope_50_200",
                     "Slope_20_80", "Slope_10_80", "Slope_10_40")
  return(res)
}

# Thin points of a particular phenotype by a factor of x.
# Points of other phenotypes will be unaffected.
randomThinPheno <- function(p, x=0.5, typesToCount=list("CD20+")) {
  pSubset <- subset(p, select=Phenotype)
  irrelevantPheno <- !(pSubset$marks %in% typesToCount)
  
  retain <- (runif(npoints(pSubset)) < x | irrelevantPheno )
  res <- p[retain]
  return(res)
}